In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')
#path = '/content/gdrive/My Drive/Colab Notebooks/707/Labs/'
#import sys
#sys.path.append(path)

In [2]:
!pip install Box2D
!pip install box2d-py
!pip install gym[all]
!pip install gym[Box_2D]
!pip install torc
!pip install -U "ray[rllib]" torch
import gym 
env = gym.make("LunarLander-v2")

  Using cached mujoco-py-1.50.1.68.tar.gz (120 kB)
  ERROR: Failed building wheel for mujoco-py
  Running setup.py clean for mujoco-py
Failed to build mujoco-py
    Running setup.py install for mujoco-py ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-nbj3qvf7/mujoco-py_8397aa03472a45909340414fb455142b/setup.py'"'"'; __file__='"'"'/tmp/pip-install-nbj3qvf7/mujoco-py_8397aa03472a45909340414fb455142b/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-_v5a1a3m/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/mujoco-py Check the logs for full command output.
     |████

In [6]:
import ray
import ray.rllib.agents.dqn as dqn

def evaluation_fn(result):
    return result['episode_reward_mean']


def objective_fn(config):

    
    trainer = dqn.DQNTrainer(config=config)

    for i in range(40):
      # Perform one iteration of training the policy with DQN
      result = trainer.train()
      intermediate_score = evaluation_fn(result)
      
      # Feed the score back back to Tune.
      tune.report(iterations=i, mean_reward=intermediate_score)
      

## Use DQN and train your algorithm on the Dungeon environment.

You can take inspiration from:
https://docs.ray.io/en/latest/rllib/rllib-training.html#basic-python-api

Experiment with the different parameters of the configuration:
https://docs.ray.io/en/latest/rllib/rllib-algorithms.html#deep-q-networks-dqn-rainbow-parametric-dqn





In [11]:

import ray
import ray.rllib.agents.dqn as dqn
from ray.tune.logger import pretty_print
from ray import tune 

config = dqn.DEFAULT_CONFIG.copy()
config["dueling"] = tune.grid_search([True, False])
config["double_q"] = tune.grid_search([True, False])
config["model"] = { "fcnet_hiddens": [128, 128, 128],
                    "fcnet_activation": 'relu',
    }
config["env"] = "LunarLander-v2"
#config['lr'] = tune.loguniform(1e-4, 1e-1),
config["gamma"] = tune.uniform(0, 1)

analysis = tune.run(
        objective_fn,
        metric="mean_reward",
        mode="max",
        num_samples=3,
        config=config)


print("Best hyperparameters found were: ", analysis.best_config)


Trial name,status,loc,double_q,dueling,gamma
objective_fn_LunarLander-v2_825a7_00000,PENDING,,True,True,0.302992
objective_fn_LunarLander-v2_825a7_00001,PENDING,,False,True,0.0945719
objective_fn_LunarLander-v2_825a7_00002,PENDING,,True,False,0.0662077
objective_fn_LunarLander-v2_825a7_00003,PENDING,,False,False,0.780091
objective_fn_LunarLander-v2_825a7_00004,PENDING,,True,True,0.914384
objective_fn_LunarLander-v2_825a7_00005,PENDING,,False,True,0.530701
objective_fn_LunarLander-v2_825a7_00006,PENDING,,True,False,0.0132863
objective_fn_LunarLander-v2_825a7_00007,PENDING,,False,False,0.907981
objective_fn_LunarLander-v2_825a7_00008,PENDING,,True,True,0.495593
objective_fn_LunarLander-v2_825a7_00009,PENDING,,False,True,0.428185


(ImplicitFunc pid=2775) 2022-04-13 12:01:58,874	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=2775) 2022-04-13 12:01:58,895	INFO trainer.py:2141 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(objective_fn pid=2775) 2022-04-13 12:01:58,895	INFO simple_q.py:155 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(objective_fn pid=2775) 2022-04-13 12:01:58,895	INFO trainer.py:781 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(ImplicitFunc pid=2769) 2022-04-13 12:01:58,872	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=2769) 2022-04-13 12:01

Trial name,status,loc,double_q,dueling,gamma
objective_fn_LunarLander-v2_825a7_00000,RUNNING,172.28.0.2:2770,True,True,0.302992
objective_fn_LunarLander-v2_825a7_00001,RUNNING,172.28.0.2:2778,False,True,0.0945719
objective_fn_LunarLander-v2_825a7_00002,RUNNING,172.28.0.2:2774,True,False,0.0662077
objective_fn_LunarLander-v2_825a7_00003,RUNNING,172.28.0.2:2775,False,False,0.780091
objective_fn_LunarLander-v2_825a7_00004,RUNNING,172.28.0.2:2776,True,True,0.914384
objective_fn_LunarLander-v2_825a7_00005,RUNNING,172.28.0.2:2773,False,True,0.530701
objective_fn_LunarLander-v2_825a7_00006,RUNNING,172.28.0.2:2772,True,False,0.0132863
objective_fn_LunarLander-v2_825a7_00007,RUNNING,172.28.0.2:2771,False,False,0.907981
objective_fn_LunarLander-v2_825a7_00008,RUNNING,172.28.0.2:2767,True,True,0.495593
objective_fn_LunarLander-v2_825a7_00009,RUNNING,172.28.0.2:2769,False,True,0.428185


(objective_fn pid=2775) 2022-04-13 12:02:00,493	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=2768) 2022-04-13 12:02:00,484	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=2772) 2022-04-13 12:02:00,495	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=2774) 2022-04-13 12:02:00,490	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=2777) 2022-04-13 12:02:00,500	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=2771) 2022-04-13 12:02:00,524	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=2769) 2022-04-13 12:02:00,793	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=2773) 2022-04-13 12:02:00,793	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=2776) 2022-04-13 12:02:00,866	WARNING util.py:55 -- Install gputil for GPU system moni

Result for objective_fn_LunarLander-v2_825a7_00011:
  date: 2022-04-13_12-02-02
  done: false
  experiment_id: f576928441b043f48c9de3d6907ccbdd
  hostname: aa9ef03c8f27
  iterations: 0
  iterations_since_restore: 1
  mean_reward: -166.21299480738392
  node_ip: 172.28.0.2
  perf:
    cpu_util_percent: 32.06666666666667
    ram_util_percent: 18.53333333333333
  pid: 2768
  time_since_restore: 3.8993098735809326
  time_this_iter_s: 3.8993098735809326
  time_total_s: 3.8993098735809326
  timestamp: 1649851322
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 825a7_00011
  
Result for objective_fn_LunarLander-v2_825a7_00007:
  date: 2022-04-13_12-02-02
  done: false
  experiment_id: fad49a1612b0497e9c1fa9f25c1b4bef
  hostname: aa9ef03c8f27
  iterations: 0
  iterations_since_restore: 1
  mean_reward: -245.5896682354995
  node_ip: 172.28.0.2
  perf:
    cpu_util_percent: 31.933333333333337
    ram_util_percent: 18.516666666666666
  pid: 2771
  time_since_restore: 3.938135862350

...

Trial name,status,loc,double_q,dueling,gamma,iter,total time (s),iterations,mean_reward
objective_fn_LunarLander-v2_825a7_00004,RUNNING,172.28.0.2:2776,True,True,0.914384,40,215.601,39,-119.314
objective_fn_LunarLander-v2_825a7_00000,TERMINATED,172.28.0.2:2770,True,True,0.302992,40,185.895,39,-104.417
objective_fn_LunarLander-v2_825a7_00001,TERMINATED,172.28.0.2:2778,False,True,0.0945719,40,184.137,39,-117.778
objective_fn_LunarLander-v2_825a7_00002,TERMINATED,172.28.0.2:2774,True,False,0.0662077,40,181.087,39,-160.45
objective_fn_LunarLander-v2_825a7_00003,TERMINATED,172.28.0.2:2775,False,False,0.780091,40,203.707,39,-115.768
objective_fn_LunarLander-v2_825a7_00005,TERMINATED,172.28.0.2:2773,False,True,0.530701,40,186.586,39,-98.0756
objective_fn_LunarLander-v2_825a7_00006,TERMINATED,172.28.0.2:2772,True,False,0.0132863,40,179.59,39,-90.4604
objective_fn_LunarLander-v2_825a7_00007,TERMINATED,172.28.0.2:2771,False,False,0.907981,40,214.946,39,-137.209
objective_fn_LunarLander-v2_825a7_00008,TERMINATED,172.28.0.2:2767,True,True,0.495593,40,192.273,39,-73.1676
objective_fn_LunarLander-v2_825a7_00009,TERMINATED,172.28.0.2:2769,False,True,0.428185,40,184.191,39,-118.152


Result for objective_fn_LunarLander-v2_825a7_00004:
  date: 2022-04-13_12-05-34
  done: true
  experiment_id: bde45c8f8c2e46b2a9a0b7a1a88b6b97
  experiment_tag: 4_double_q=True,dueling=True,gamma=0.91438
  hostname: aa9ef03c8f27
  iterations: 39
  iterations_since_restore: 40
  mean_reward: -119.31391653395582
  node_ip: 172.28.0.2
  perf:
    cpu_util_percent: 6.199999999999999
    ram_util_percent: 9.4
  pid: 2776
  time_since_restore: 215.60084104537964
  time_this_iter_s: 4.511471509933472
  time_total_s: 215.60084104537964
  timestamp: 1649851534
  timesteps_since_restore: 0
  training_iteration: 40
  trial_id: 825a7_00004
  


Trial name,status,loc,double_q,dueling,gamma,iter,total time (s),iterations,mean_reward
objective_fn_LunarLander-v2_825a7_00000,TERMINATED,172.28.0.2:2770,True,True,0.302992,40,185.895,39,-104.417
objective_fn_LunarLander-v2_825a7_00001,TERMINATED,172.28.0.2:2778,False,True,0.0945719,40,184.137,39,-117.778
objective_fn_LunarLander-v2_825a7_00002,TERMINATED,172.28.0.2:2774,True,False,0.0662077,40,181.087,39,-160.45
objective_fn_LunarLander-v2_825a7_00003,TERMINATED,172.28.0.2:2775,False,False,0.780091,40,203.707,39,-115.768
objective_fn_LunarLander-v2_825a7_00004,TERMINATED,172.28.0.2:2776,True,True,0.914384,40,215.601,39,-119.314
objective_fn_LunarLander-v2_825a7_00005,TERMINATED,172.28.0.2:2773,False,True,0.530701,40,186.586,39,-98.0756
objective_fn_LunarLander-v2_825a7_00006,TERMINATED,172.28.0.2:2772,True,False,0.0132863,40,179.59,39,-90.4604
objective_fn_LunarLander-v2_825a7_00007,TERMINATED,172.28.0.2:2771,False,False,0.907981,40,214.946,39,-137.209
objective_fn_LunarLander-v2_825a7_00008,TERMINATED,172.28.0.2:2767,True,True,0.495593,40,192.273,39,-73.1676
objective_fn_LunarLander-v2_825a7_00009,TERMINATED,172.28.0.2:2769,False,True,0.428185,40,184.191,39,-118.152


2022-04-13 12:05:34,919	INFO tune.py:639 -- Total run time: 221.23 seconds (220.83 seconds for the tuning loop).


Best hyperparameters found were:  {'num_workers': 0, 'num_envs_per_worker': 1, 'create_env_on_driver': False, 'rollout_fragment_length': 4, 'batch_mode': 'truncate_episodes', 'gamma': 0.5616905067242305, 'lr': 0.0005, 'train_batch_size': 32, 'model': {'fcnet_hiddens': [128, 128, 128], 'fcnet_activation': 'relu'}, 'optimizer': {}, 'horizon': None, 'soft_horizon': False, 'no_done_at_end': False, 'env': 'LunarLander-v2', 'observation_space': None, 'action_space': None, 'env_config': {}, 'remote_worker_envs': False, 'remote_env_batch_wait_ms': 0, 'env_task_fn': None, 'render_env': False, 'record_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'preprocessor_pref': 'deepmind', 'log_level': 'WARN', 'callbacks': <class 'ray.rllib.agents.callbacks.DefaultCallbacks'>, 'ignore_worker_failures': False, 'log_sys_usage': True, 'fake_sampler': False, 'framework': 'tf', 'eager_tracing': False, 'eager_max_retraces': 20, 'explore': True, 'exploration_config': {'type'

In [12]:
df = analysis.dataframe(metric="mean_reward", mode="max")
df[['config/gamma', 'config/double_q', 'config/dueling', 'mean_reward']]

,config/gamma,config/double_q,config/dueling,mean_reward
0,0.302992,True,True,-102.627613
1,0.094572,False,True,-117.778401
2,0.066208,True,False,-136.565895
3,0.780091,False,False,-114.242969
4,0.914384,True,True,-119.313917
5,0.530701,False,True,-98.075567
6,0.013286,True,False,-90.460400
7,0.907981,False,False,-135.542733
8,0.495593,True,True,-71.917700
9,0.428185,False,True,-118.152430
